In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

import pandas as pd
from collections import OrderedDict

import pprint
import re
import numpy as np
import matplotlib.pyplot as plt
import random

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)  
import glob
import uproot 
import ROOT

import platform
print('Python:', platform.python_version())

Welcome to JupyROOT 6.26/06
Python: 3.10.8


In [2]:
run = 10645

filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_pedestal-*{run}-0000.prdf.root")
filenames.sort()
pprint.pprint(filenames)

['/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc00_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc01_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc03_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc04_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc05_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc06_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc07_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc08_pedestal-00010645-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_eb

In [3]:
files = dict()
for filename in filenames: 
    file = ROOT.TFile.Open(filename)
    sector = re.match(".*ebdc([0-9][0-9])_.*", filename).group(1)
    files[sector] = file


pprint.pprint(files);

{'00': <cppyy.gbl.TFile object at 0x7ff5240>,
 '01': <cppyy.gbl.TFile object at 0x66dab40>,
 '02': <cppyy.gbl.TFile object at 0x83ce530>,
 '03': <cppyy.gbl.TFile object at 0x8095000>,
 '04': <cppyy.gbl.TFile object at 0x8347af0>,
 '05': <cppyy.gbl.TFile object at 0x83cd850>,
 '06': <cppyy.gbl.TFile object at 0x8354080>,
 '07': <cppyy.gbl.TFile object at 0x83bd7a0>,
 '08': <cppyy.gbl.TFile object at 0x83cfe40>,
 '09': <cppyy.gbl.TFile object at 0x83d2b60>,
 '10': <cppyy.gbl.TFile object at 0x82b12e0>,
 '11': <cppyy.gbl.TFile object at 0x82b2380>,
 '12': <cppyy.gbl.TFile object at 0x841a020>,
 '13': <cppyy.gbl.TFile object at 0x841cc50>,
 '14': <cppyy.gbl.TFile object at 0x841f9e0>,
 '15': <cppyy.gbl.TFile object at 0x8432840>,
 '16': <cppyy.gbl.TFile object at 0x84355e0>,
 '17': <cppyy.gbl.TFile object at 0x84382d0>,
 '18': <cppyy.gbl.TFile object at 0x843b070>,
 '19': <cppyy.gbl.TFile object at 0x843dbe0>,
 '20': <cppyy.gbl.TFile object at 0x843f570>,
 '21': <cppyy.gbl.TFile object at 

In [4]:
files['00'].ls()
files['00'].PacketTree.Show(0)
files['00'].SampleTree.Show(0)
files['00'].TaggerTree.Show(0)

TFile**		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc00_pedestal-00010645-0000.prdf.root	
 TFile*		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc00_pedestal-00010645-0000.prdf.root	
  KEY: TTree	SampleTree;2	Each entry is one waveform [current cycle]
  KEY: TTree	SampleTree;1	Each entry is one waveform [backup cycle]
  KEY: TTree	PacketTree;1	Each entry is one packet
  KEY: TTree	TaggerTree;1	Each entry is one tagger for level 1 trigger or endat tag
======> EVENT:0
 packet          = 4000
 frame           = 2
 nWaveormInFrame = 78
 nTaggerInFrame  = 2
 maxFEECount     = 26
======> EVENT:0
 packet          = 4000
 frame           = 2
 nWaveormInFrame = 78
 maxFEECount     = 26
 nSamples        = 360
 adcSamples      = 78, 
                  79, 75, 78, 78, 82, 80, 78, 75, 78, 82, 
                  82, 76, 74, 80, 81, 78, 75, 76, 77
 fee             = 11
 sampaAddress    = 1
 sampaChannel    = 8
 Channel         = 40
 BCO             = 564491
 check

In [5]:
%%cpp -d

#include<sPhenixStyle.C>

In [6]:
%%cpp

SetsPhenixStyle()

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [7]:
%jsroot on

In [8]:
c1 = ROOT.TCanvas( 'checksumErrorQA', 'checksumErrorQA', 1500, 1200)

c1.Divide(4,3)

i = 1
for sector, file in files.items():
    
    c1.cd(i)
    i=i+1
    
    hAllWaveForm = ROOT.TH1F(f'hAllWaveForm{sector}', ';fee*8+sampaAddress [0..207]', 26*8, -.5, 26*8 -.5);
    hErrorWaveForm = ROOT.TH1F(f'hErrorWaveForm{sector}', f';Sector{sector}: fee*8+sampaAddress [0..207];Probability of CheckSum Error', 26*8, -.5, 26*8 -.5);

    files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hAllWaveForm{sector}","","goff");
    files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hErrorWaveForm{sector}","checksumError>0","goff");
    
    hErrorWaveForm.Divide (hErrorWaveForm, hAllWaveForm, 1, 1, 'B');
    hErrorWaveForm.DrawClone();

c1.Draw()

In [10]:

c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1500, 1200)

c1.Divide(4,3)

i = 1
for sector, file in files.items():
    

    pad = c1.cd(i)
    i=i+1

    pad.SetLogz()
    
    hADC = ROOT.TH2F(f'hADC{sector}', f';Sector{sector}: ADC Time bin [1/20MHz]; ADC [ADU]', 360, 0, 360, 256,0,1024);

    files[sector].SampleTree.Draw(f"adcSamples:Iteration$>>hADC{sector}","checksumError==0","colz");
    
    hADC.DrawClone("colz");
    
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ADCQA
Warning in <TFile::Append>: Replacing existing TH1: hADC00 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC01 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC02 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC03 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC04 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC05 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC06 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC07 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC08 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC09 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADC10 (Potential memory leak).
War